In [3]:
using JuMP, Gurobi

In [56]:
# Make checkerboard as an example, with one shiften one column to the right
origin = zeros(Int32, 12, 12)
origin[2:2:11, 2:2:10] .= 1
origin[3:2:11, 3:2:10] .= 1

destination = zeros(Int32, 12, 12)
destination[2:2:11, 3:2:11] .= 1
destination[3:2:11, 4:2:11] .= 1

origin[1, :] = origin[12, :] = origin[:, 12] = origin[:, 1] .= 1
destination[1,:]=destination[12,:]=destination[:,12]=destination[:,1] .= 1
;

12×12 Matrix{Int32}:
 1  1  1  1  1  1  1  1  1  1  1  1
 1  0  1  0  1  0  1  0  1  0  1  1
 1  0  0  1  0  1  0  1  0  1  0  1
 1  0  1  0  1  0  1  0  1  0  1  1
 1  0  0  1  0  1  0  1  0  1  0  1
 1  0  1  0  1  0  1  0  1  0  1  1
 1  0  0  1  0  1  0  1  0  1  0  1
 1  0  1  0  1  0  1  0  1  0  1  1
 1  0  0  1  0  1  0  1  0  1  0  1
 1  0  1  0  1  0  1  0  1  0  1  1
 1  0  0  1  0  1  0  1  0  1  0  1
 1  1  1  1  1  1  1  1  1  1  1  1

In [64]:
function getDistance(img1, img2, ws, epsilon)
    image_width, image_height = size(img1)
    xIter, yIter, fIter = 0:(image_width-1), 0:(image_height-1), 0:(2*ws)
    
    m = direct_model(Gurobi.Optimizer())
    
    # Variable for each point, and each potential destination of each point
    @variable(m, v[xIter, yIter, fIter, fIter], Bin)
    
    # Objective to minimize change of color between img1 and img2 with epsilon to minimize total change
    @objective(m, Min, sum(
            (img1[x+1, y+1] - img2[x + 1 + (xs - ws), y + 1 + (ys  - ws)])^2 * v[x, y, xs, ys]
            for x in xIter, y in yIter, xs in fIter, ys in fIter
            if x + (xs - ws) >= 0 &&
               x + (xs - ws) < image_width &&
               y + (ys - ws) >= 0 &&
               y + (ys - ws) < image_height
        ) + epsilon * sum(v)
    )
    
    column(x) = Cint(Gurobi.column(backend(m), index(x)) - 1) # Magic function to call Gurobi C-layer API with variables by index
    
    function addLeftConstraint(x, y, xs, ys)
        if xs != 2*ws && xs != 2*ws-1 # left pixel can't be more right than curr pixel
            max1 = @variable(m, binary=true)
            max2 = @variable(m, binary=true)
            max3 = @variable(m, binary=true)
            v1 = [v[x,y,xss,yss] for xss in 0:xs, yss in fIter]
            v2 = [v[x-1,y,xss,yss]  for xss in (xs+2):(2*ws), yss in fIter]
            v3 = [max1, max2]
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max1), length(v1), column.(v1), 0)
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max2), length(v2), column.(v2), 0)
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max3), length(v3), column.(v3), 0)
            @constraint(m, max1 + max2 <= max3)
        end
    end
    
    function addDownConstraint(x, y, xs, ys)
        if ys != 2*ws && ys != 2*ws-1
            max1 = @variable(m, binary=true)
            max2 = @variable(m, binary=true)
            max3 = @variable(m, binary=true)
            v1 = [v[x,y,xss,yss] for xss in fIter, yss in 0:ys]
            v2 = [v[x,y-1,xss,yss] for xss in fIter, yss in (ys+2):(2*ws)]
            v3 = [max1, max2]
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max1), length(v1), column.(v1), 0)
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max2), length(v2), column.(v2), 0)
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max3), length(v3), column.(v3), 0)
            @constraint(m, max1 + max2 <= max3)
        end
    end
    
    function addDownLeftConstraint(x, y, xs, ys)
        if xs != 2*ws &&  xs != 2*ws-1
            max1 = @variable(m, binary=true)
            max2 = @variable(m, binary=true)
            max3 = @variable(m, binary=true)
            v1 = [v[x,y,xss,yss] for xss in 0:xs, yss in fIter]
            v2 = [v[x-1,y-1,xss,yss] for xss in (xs+2):(2*ws), yss in fIter]
            v3 = [max1, max2]
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max1), length(v1), column.(v1), 0)
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max2), length(v2), column.(v2), 0)
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max3), length(v3), column.(v3), 0)
            @constraint(m, max1 + max2 <= max3)
        end
        if ys != 2*ws && ys != 2*ws-1
            max1 = @variable(m, binary=true)
            max2 = @variable(m, binary=true)
            max3 = @variable(m, binary=true)
            v1 = [v[x,y,xss,yss] for xss in fIter, yss in 0:ys]
            v2 = [v[x-1,y-1,xss,yss] for xss in fIter, yss in (ys+2):2*ws]
            v3 = [max1, max2]
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max1), length(v1), column.(v1), 0)
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max2), length(v2), column.(v2), 0)
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max3), length(v3), column.(v3), 0)
            @constraint(m, max1 + max2 <= max3)
        end
    end
    
    function addUpLeftConstraint(x, y, xs, ys)
        if xs != 2*ws && xs != 2*ws-1
            max1 = @variable(m, binary=true)
            max2 = @variable(m, binary=true)
            max3 = @variable(m, binary=true)
            v1 = [v[x,y,xss,yss] for xss in 0:xs, yss in fIter]
            v2 = [v[x-1,y+1,xss,yss] for xss in (xs+2):(2*ws), yss in fIter]
            v3 = [max1, max2]
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max1), length(v1), column.(v1), 0)
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max2), length(v2), column.(v2), 0)
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max3), length(v3), column.(v3), 0)
            @constraint(m, max1 + max2 <= max3)
        end
        if ys != 0 && ys != 1
            max1 = @variable(m, binary=true)
            max2 = @variable(m, binary=true)
            max3 = @variable(m, binary=true)
            v1 = [v[x,y,xss,yss] for xss in fIter, yss in ys:(2*ws)]
            v2 = [v[x-1,y+1,xss,yss] for xss in fIter, yss in 0:(ys-2)]
            v3 = [max1, max2]
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max1), length(v1), column.(v1), 0)
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max2), length(v2), column.(v2), 0)
            Gurobi.GRBaddgenconstrMax(backend(m), "", column(max3), length(v3), column.(v3), 0)
            @constraint(m, max1 + max2 <= max3)
        end
    end
    
    # add constraints that if a pixel shift is outside image, it must be 0
    isOutside(x, xs, h) = x + (xs - ws) < 0 || x + (xs - ws) > h - 1
    for x in xIter, y in yIter, xs in fIter, ys in fIter
        if  isOutside(x, xs, image_width) || isOutside(y, ys, image_height)
            @constraint(m, v[x, y, xs, ys] == 0)
        end
    end
    
    # constraints that all source pixels must shift in at least 1 direction
    @constraint(m, mustShiftC[x in xIter, y in yIter], sum(v[x,y,:,:]) >= 1)
    
    
    # constraints that all dest pixels must be mapped to
    isInside(x, xs, h) = x - (xs - ws) >= 0 && x - (xs - ws) < h
    @constraint(m, ontoC[x in xIter, y in yIter],
        sum(v[x - (xs - ws), y - (ys - ws), xs, ys] for xs in fIter, ys in fIter
                if isInside(x, xs, image_width) && isInside(y, ys, image_height)
            ) >= 1
    )
    
    for x in xIter, y in yIter, xs in fIter, ys in fIter
        # Set initial values
        if xs == ws && ys == ws
            set_start_value(v[x, y, xs, ys], 1)
        else
            set_start_value(v[x, y, xs, ys], 0)
        end
        
        # No part constraint
        for xn in max(0, x-1):min(image_width - 1, x+1), yn in max(0,y-1):min(image_height - 1, y+1)
            if xn != x || yn != y
                @constraint(m,
                    v[x,y,xs,ys] <= sum(v[xn, yn, xss, yss] for xss in max(0, xs-1):min(2*ws, xs+1), yss in max(0,ys-1):min(2*ws,ys+1))
                )
            end
        end
        if y == 0 && x == 0
            continue
        elseif y == 0 # Left constraint
            addLeftConstraint(x, y, xs, ys)
        elseif x == 0 # Down constraint
            addDownConstraint(x, y, xs, ys)
        else
            addLeftConstraint(x, y, xs, ys)
            addDownConstraint(x, y, xs, ys)
            addDownLeftConstraint(x, y, xs, ys)
        end
        if y != image_height-1 && x != 0
            addUpLeftConstraint(x, y, xs, ys)
        end
    end
    
    return m
end

getDistance (generic function with 1 method)

In [65]:
m = getDistance(origin, destination, 1, 0.01);

optimize!(m)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-29
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD Ryzen 7 3800X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 11780 rows, 8028 columns and 68020 nonzeros
Model fingerprint: 0xa1d486a0
Model has 6732 general constraints
Variable types: 0 continuous, 8028 integer (8028 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

User MIP start produced solution with objective 91.44 (0.01s)
Loaded user MIP start with objective 91.44

Presolve added 2132 rows and 0 columns
Presolve removed 0 rows and 5376 columns
Presolve time: 0.23s
Presolved: 13912 rows, 2652 columns, 59868 nonzeros
Variable types: 0 continuous, 2652 integer (2652 binary)
Root relaxation presolved: